In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_seq_items', 1000)
%matplotlib inline


## Call atlas data

In [2]:
df = pd.read_csv('../../data/external/postcodes.csv')

/Users/harry/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,17,18,21,22,24,25,29,32,39,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df[['Ward', 'Ward Code', 'Latitude', 'Longitude', 'Population',
       'Index of Multiple Deprivation',
       'Average Income', 'Region']].dropna(subset=['Ward', 'Ward Code'])

In [4]:
df.isnull().sum(axis = 0)

Ward                                   0
Ward Code                              0
Latitude                               0
Longitude                              0
Population                       1129958
Index of Multiple Deprivation          0
Average Income                    283943
Region                            422618
dtype: int64

In [5]:
df.shape

(2620747, 8)

In [6]:
df.columns

Index(['Ward', 'Ward Code', 'Latitude', 'Longitude', 'Population',
       'Index of Multiple Deprivation', 'Average Income', 'Region'],
      dtype='object')

## Group data by ward code

In [7]:
location_df = (df.groupby(['Ward Code', 'Ward'], as_index=False)
   .agg({'Region':'first', 'Latitude':'first', 'Longitude':'first', 'Population':'mean', 'Index of Multiple Deprivation':'mean',
         'Average Income':'mean'}))

In [8]:
location_df.isnull().sum(axis = 0)

Ward Code                           0
Ward                                0
Region                           1668
Latitude                            0
Longitude                           0
Population                          0
Index of Multiple Deprivation       0
Average Income                    816
dtype: int64

In [9]:
location_df.shape

(8815, 8)

## Make ward atlas data

In [10]:
ward_df = pd.read_excel('../../data/external/ward-results.xlsx').dropna(subset=['Remain', 'Leave'])

In [11]:
ward_df.shape


(1283, 8)

In [12]:
location_df.rename(columns={"Ward Code": "WardCode"}, inplace=True)

In [13]:
ward_atlas_df = (pd.merge(location_df, ward_df, how='inner', on='WardCode')
                   .drop(columns=['WardName', 'CountingArea', 'Postals', 'Remain', 'Leave'])
                   .dropna(subset=['Average Income']))

In [14]:
ward_atlas_df['Leave?'] = (ward_atlas_df['Leave%'] >= ward_atlas_df['Remain%']).astype(int)

In [15]:
ward_atlas_df['Deficitsq'] = ((ward_atlas_df['Leave%'] - ward_atlas_df['Remain%'])**2).astype(float)

In [16]:
ward_atlas_df['Deficit'] = (abs(ward_atlas_df['Leave%'] - ward_atlas_df['Remain%'])).astype(float)

In [17]:
ward_atlas_df.head()

,WardCode,Ward,Region,Latitude,Longitude,Population,Index of Multiple Deprivation,Average Income,Remain%,Leave%,Leave?,Deficitsq,Deficit
0,E05000106,Bickley,London,51.403386,0.043619,43.231429,26731.237245,65522.448980,55.741321,44.258679,0,131.851079,11.482643
1,E05000107,Biggin Hill,London,51.319163,0.009060,49.093596,23854.819780,59529.670330,35.302391,64.697609,1,864.078841,29.395218
2,E05000108,Bromley Common and Keston,London,51.350450,0.034039,51.776632,21342.703608,59368.041237,47.217299,52.782701,1,30.973703,5.565402
3,E05000109,Bromley Town,London,51.401546,0.015415,48.155620,17205.781723,57839.813737,57.344262,42.655738,0,215.752755,14.688525
4,E05000110,Chelsfield and Pratts Bottom,London,51.369232,0.136792,43.411940,27848.258145,60107.017544,45.922747,54.077253,1,66.495975,8.154506


In [18]:
ward_atlas_df.shape

(1059, 13)

In [19]:
ward_atlas_df.isnull().sum(axis = 0)

WardCode                         0
Ward                             0
Region                           0
Latitude                         0
Longitude                        0
Population                       0
Index of Multiple Deprivation    0
Average Income                   0
Remain%                          0
Leave%                           0
Leave?                           0
Deficitsq                        0
Deficit                          0
dtype: int64

In [20]:
ward_atlas_df.to_csv('ward_atlas_data.csv')